In [1]:
import os
import cv2
import mediapipe as mp
import pandas as pd
from tqdm import tqdm

In [2]:
# --- MediaPipe setup ---
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1)

In [3]:
# --- Normalize landmarks relative to the wrist (landmark 0) ---
def normalize_landmarks(landmarks):
    origin_x = landmarks[0].x
    origin_y = landmarks[0].y
    origin_z = landmarks[0].z

    normalized = []
    for lm in landmarks:
        normalized.extend([
            lm.x - origin_x,
            lm.y - origin_y,
            lm.z - origin_z
        ])
    return normalized

In [4]:
# --- Extract and normalize from image ---
def extract_landmarks(image_path):
    image = cv2.imread(image_path)
    if image is None:
        return None

    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image_rgb)

    if results.multi_hand_landmarks:
        landmarks = results.multi_hand_landmarks[0].landmark
        return normalize_landmarks(landmarks)  # 👈 normalize here
    else:
        return None

In [5]:
# --- Dataset processing ---
dataset_path = "../dataset/Train_Alphabet"
data = []
labels = []

# Process folders A-Z
classes = sorted(os.listdir(dataset_path))[:26]  # A-Z

for label in tqdm(classes):
    folder_path = os.path.join(dataset_path, label)
    images = os.listdir(folder_path)[:300]  # Limit per class for speed

    for img_name in images:
        img_path = os.path.join(folder_path, img_name)
        landmarks = extract_landmarks(img_path)

        if landmarks:
            data.append(landmarks)
            labels.append(label)

100%|██████████████████████████████████████████████████████████████████████████████████| 26/26 [16:40<00:00, 38.47s/it]


In [6]:
# --- Save to CSV ---
df = pd.DataFrame(data)
df['label'] = labels
df.to_csv("../dataset/normalized_landmark_data.csv", index=False)
print("Normalized landmark dataset saved as normalized_landmark_data.csv")

Normalized landmark dataset saved as normalized_landmark_data.csv
